Imports for project

In [ ]:
import json
import tqdm
import dask.dataframe as dd

FunSoCs Predictions Table

In [1]:

funsocs_dataframe = dd.read_csv("/media/felix/ShortTerm/funsocs_20.9.26.tsv", sep="\t")
funsocs_df = funsocs_dataframe[["Entry", "cytotoxicity"]].compute()
funsocs_data = funsocs_df["Entry"].tolist()

FunSoCs ground truth labels

In [3]:
cytotoxic_ground_truth = open("/run/user/1000/gvfs/smb-share:server=truenas.local,share=research/FunSoCTrainingData/Cytotoxicity_v1.0_10.24.2019.txt").read().splitlines()

Collecting UniRef Clusters at specified identity. 

In [9]:
import Bio.SeqIO as seqio
unirefclust_records = seqio.parse("/media/felix/ShortTerm/uniprotkb_uniref_cluster_90_UniRef90_P7_2024_03_06_cleaned.fasta","fasta") # we loose 5 becuase of cleaning


In [10]:
import tqdm
entry_dict = {}
cluster_representative_id = "P77335"
for entry in tqdm.tqdm( unirefclust_records ):
    if str(entry.id).split("|")[1] in cytotoxic_ground_truth:
        entry_dict[str(entry.id).split("|")[1]] = 1
        
print(entry_dict)

142it [00:00, 8859.80it/s]

{'P77335': 1, 'Q8Z727': 1, 'Q9REB3': 1}


In [4]:
def check_entry_parent(entry):
    """"
    Returns the parent entry for a given entry
    """
    uniref = json.load(open("/media/felix/ShortTerm/unirefclust_50.json","r"))
    for member in uniref["members"]:
        if member["accessions"][0] == str(entry.id).split("|")[1]:
            return member["uniref100Id"].split("_")[1]
    return None

Check for labeled data. 
1. Check if ground truth exist for given entry
2. If none is available check prediction
3. If prediction is available, check if prediction's uniref100 clust representative matches your 90 representative. 
4. If not, check current unirefclust rep of entry
5. If current rep doesn't have a prediction, run seqscreen

In [27]:
import tqdm
entry_dict = {}
cluster_representative_id = "P77335"
unirefclust_records = seqio.parse("/media/felix/ShortTerm/uniprotkb_uniref_cluster_50_UniRef50_P7_2024_03_06_cleaned.fasta","fasta") # we loose 5 becuase of cleaning

for entry in tqdm.tqdm( unirefclust_records ):
    if str(entry.id).split("|")[1] in cytotoxic_ground_truth:
        entry_dict[str(entry.id).split("|")[1]] = 1
    elif str(entry.id).split("|")[1] in funsocs_data:
        # lookup 100% identity parent
        if check_entry_parent(entry) == cluster_representative_id:
            
                entry_dict[str(entry.id).split("|")[1]] = int(funsocs_df[funsocs_df ["Entry"]==str(entry.id).split("|")[1]]["cytotoxicity"].iloc[0])
        else:
            if cluster_representative_id in cytotoxic_ground_truth:
                entry_dict[str(entry.id).split("|")[1]] = 0.5
            elif cluster_representative_id in funsocs_data and check_entry_parent(cluster_representative_id) == cluster_representative_id:
                entry_dict[str(entry.id).split("|")[1]] = int(funsocs_df[funsocs_df ["Entry"]==cluster_representative_id]["cytotoxicity"].iloc[0])
            else:
                print("Run seqscreen on representative")
            
    else:
        if cluster_representative_id in cytotoxic_ground_truth:
            entry_dict[str(entry.id).split("|")[1]] = 0.5
        elif cluster_representative_id in funsocs_data and check_entry_parent(cluster_representative_id) == cluster_representative_id:
            entry_dict[str(entry.id).split("|")[1]] = int(funsocs_df[funsocs_df ["Entry"]==cluster_representative_id]["cytotoxicity"].iloc[0])
        else:
            print("Run seqscreen on representative")
        

249it [01:00,  4.11it/s]


In [31]:
cluster_representative_id = "P77335"
funsocs_df[funsocs_df ["Entry"]=="Q68S90"]

,Entry,cytotoxicity
844328,Q68S90,0


In [28]:
entry_dict

{'P77335': 1,
 'Q8Z727': 1,
 'Q93RR6': 1,
 'Q9REB3': 1,
 'A0A080J6B5': 0.5,
 'A0A085AIJ6': 0.5,
 'A0A0H0K1M2': 0.5,
 'A0A0H3MHU8': 0.5,
 'A0A0K9T6X3': 0.5,
 'A0A0N1R1F1': 0.5,
 'A0A1H9EXJ2': 0.5,
 'A0A1Q5ZQ68': 0.5,
 'A0A1Q6BEN5': 0.5,
 'A0A1S0ZLZ6': 0.5,
 'A0A1S9IWH5': 0.5,
 'A0A1X3I9Z5': 0.5,
 'A0A1X3JJK2': 0.5,
 'A0A1X3L5H3': 0.5,
 'A0A246NSJ8': 0.5,
 'A0A2A6DF26': 0.5,
 'A0A2A6Q0N8': 0.5,
 'A0A2I5HGP9': 0.5,
 'A0A2S8I3U1': 0.5,
 'A0A2T3V193': 0.5,
 'A0A2X1NHM9': 0.5,
 'A0A2X5H3H2': 0.5,
 'A0A2Y8JUS3': 0.5,
 'A0A315G6H4': 0.5,
 'A0A376X1I8': 0.5,
 'A0A379TZ51': 0.5,
 'A0A398QCK9': 0.5,
 'A0A3F3J4I9': 0.5,
 'A0A3J4Y844': 0.5,
 'A0A3K6VT88': 0.5,
 'A0A3K9ERY3': 0.5,
 'A0A3Q9LER3': 0.5,
 'A0A3Q9MAQ9': 0.5,
 'A0A3R0QE12': 0.5,
 'A0A3R0X038': 0.5,
 'A0A3R0X6L8': 0.5,
 'A0A3R0YBZ1': 0.5,
 'A0A3T2YQT6': 0.5,
 'A0A3T5HHB7': 0.5,
 'A0A3U2TNT3': 0.5,
 'A0A3U6WFK9': 0.5,
 'A0A3U8JES5': 0.5,
 'A0A3V3Z7S6': 0.5,
 'A0A3V4R287': 0.5,
 'A0A3V5UK95': 0.5,
 'A0A3V6QHC2': 0.5,
 'A0A3V7IP46': 0.5,
 'A0